In [1]:
import os
dll_directory = r'C:\Users\beyza\anaconda3\Lib\site-packages\openslide\openslide-win64-20230414\bin'
os.add_dll_directory(dll_directory)

<AddedDllDirectory('C:\\Users\\beyza\\anaconda3\\Lib\\site-packages\\openslide\\openslide-win64-20230414\\bin')>

In [2]:
import openslide
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import csv
import xml.etree.ElementTree as ET
import cv2
from PIL import Image, ImageFile
from openslide import OpenSlide
from patchify import patchify
from PIL.Image import DecompressionBombWarning
import warnings

In [3]:
file_paths = [
    "D:/AIN3007_project/external_test_data/",
    "D:/AIN3007_project/internal_test_data/"]

all_slides = []
all_annotations = []

for file_path in file_paths:
    # List all files in file
    for root, dirs, files in os.walk(file_path):
        for file in files:
            # Check the extension of the file.
            if file.lower().endswith((".svs", ".tiff", "tif", ".ndpi")):
                # Save the slides path in all_slides.
                file_full_path = os.path.join(root, file)
                all_slides.append(f'{file_full_path}')

for file_path in file_paths:
    # List all files in file
    for root, dirs, files in os.walk(file_path):
        for file in files:
            # Check the extension of the file.
            if file.lower().endswith((".xml")):
                # Save the slides path in all_slides.
                file_full_path = os.path.join(root, file)
                all_annotations.append(f'{file_full_path}')

# Specify the output file paths
annotations_file_path = "C:\\Users\\beyza\\Desktop\\AIN3007_project\\test_annotations.txt"
slides_file_path = "C:\\Users\\beyza\\Desktop\\AIN3007_project\\test_slides.txt"

with open(slides_file_path, "w") as slides_file:
    for slide_path in all_slides:
        slides_file.write(slide_path + "\n")

with open(annotations_file_path, "w") as annotations_file:
    for annotation_path in all_annotations:
        annotations_file.write(annotation_path + "\n")


In [4]:
# Dosya yollarını tanımlayın
annotations_file_path = "C:\\Users\\beyza\\Desktop\\AIN3007_project\\test_annotations.txt"
slides_file_path = "C:\\Users\\beyza\\Desktop\\AIN3007_project\\test_slides.txt"

# Verileri dosyalardan okuyun
with open(annotations_file_path, "r") as annotations_file:
    annotations_data = annotations_file.readlines()

with open(slides_file_path, "r") as slides_file:
    slides_data = slides_file.readlines()

# CSV dosyasına yazmak için yeni bir dosya açın ve verileri yazın
csv_file_path = "C:\\Users\\beyza\\Desktop\\AIN3007_project\\test_annotations_and_slides.csv"

# Verileri birleştirin
combined_data = zip(slides_data, annotations_data)

# CSV dosyasına verileri yazın
with open(csv_file_path, "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Slides", "Annotations"])  # Başlık satırını yazın (isteğe bağlı)
    csv_writer.writerows(combined_data)

print(f"CSV dosyası başarıyla oluşturuldu: {csv_file_path}")


CSV dosyası başarıyla oluşturuldu: C:\Users\beyza\Desktop\AIN3007_project\test_annotations_and_slides.csv


In [5]:
df = pd.read_csv(csv_file_path)
df.head()

,Slides,Annotations
0,D:/AIN3007_project/external_test_data/breast__...,D:/AIN3007_project/external_test_data/breast__...
1,D:/AIN3007_project/external_test_data/breast__...,D:/AIN3007_project/external_test_data/breast__...
2,D:/AIN3007_project/external_test_data/breast__...,D:/AIN3007_project/external_test_data/breast__...
3,D:/AIN3007_project/external_test_data/breast__...,D:/AIN3007_project/external_test_data/breast__...
4,D:/AIN3007_project/external_test_data/breast__...,D:/AIN3007_project/external_test_data/breast__...


In [6]:
df = pd.read_csv("C:\\Users\\beyza\\Desktop\\AIN3007_project\\test_slides.txt", header=None, names=['Slides'])

# Adım 2: Slaytlar üzerinde döngü yapın ve boyutları alın
for index, row in df.iterrows():
    wsi_path = row['Slides']
    try:
        slide = openslide.OpenSlide(wsi_path)
        level_dims = slide.level_dimensions
        first_level_dims = level_dims[0]  # İlk seviye boyutlarını al
        df.at[index, 'image_shape'] = str(first_level_dims)
    except Exception as e:
        print(f"Error opening {wsi_path}: {e}")
        df.at[index, 'image_shape'] = "Error"

# Adım 3: Sonuçları CSV dosyasına yazdırın
csv_file_path = "C:\\Users\\beyza\\Desktop\\AIN3007_project\\test_annotations_and_slides.csv"
df.shape

(100, 2)

In [7]:
annotation_yolu = "C:\\Users\\beyza\\Desktop\\AIN3007_project\\test_annotations.txt"
annotation_yollari = [
    "D:/AIN3007_project/internal_test_data/breast__he__10660.xml",
    "D:/AIN3007_project/internal_test_data/breast__he__11702.xml",
    "D:/AIN3007_project/internal_test_data/breast__he__12414.xml",
    "D:/AIN3007_project/internal_test_data/breast__he__13231.xml",
    "D:/AIN3007_project/internal_test_data/breast__he__13937.xml",
    "D:/AIN3007_project/internal_test_data/breast__he__14415.xml",
    "D:/AIN3007_project/internal_test_data/breast__he__14423.xml",
    "D:/AIN3007_project/internal_test_data/breast__he__14720.xml",
    "D:/AIN3007_project/internal_test_data/breast__he__15281.xml",
    "D:/AIN3007_project/internal_test_data/breast__he__18803.xml"
]

for annotation_yolu in annotation_yollari:
    try:
        os.remove(annotation_yolu)
        print(f"{annotation_yolu} başarıyla silindi.")
    except FileNotFoundError:
        print(f"{annotation_yolu} dosyası bulunamadı.")
    except Exception as e:
        print(f"{annotation_yolu} dosyası silinirken bir hata oluştu: {str(e)}")


D:/AIN3007_project/internal_test_data/breast__he__10660.xml dosyası bulunamadı.
D:/AIN3007_project/internal_test_data/breast__he__11702.xml dosyası bulunamadı.
D:/AIN3007_project/internal_test_data/breast__he__12414.xml dosyası bulunamadı.
D:/AIN3007_project/internal_test_data/breast__he__13231.xml dosyası bulunamadı.
D:/AIN3007_project/internal_test_data/breast__he__13937.xml dosyası bulunamadı.
D:/AIN3007_project/internal_test_data/breast__he__14415.xml dosyası bulunamadı.
D:/AIN3007_project/internal_test_data/breast__he__14423.xml dosyası bulunamadı.
D:/AIN3007_project/internal_test_data/breast__he__14720.xml dosyası bulunamadı.
D:/AIN3007_project/internal_test_data/breast__he__15281.xml dosyası bulunamadı.
D:/AIN3007_project/internal_test_data/breast__he__18803.xml dosyası bulunamadı.


In [8]:
with open("C:\\Users\\beyza\\Desktop\\AIN3007_project\\test_annotations.txt", "r") as file:
    annotations = file.readlines()  # Her satırı bir liste elemanı olarak okur.

# Satır sonu karakterlerini kaldırın (\n)
annotations = [line.strip() for line in annotations]
print(len(annotations))
# Annotations listesini DataFrame'e 'annotations' adlı yeni bir sütun olarak ekleyin.
df['annotations'] = annotations
# Güncellenmiş DataFrame'i CSV olarak kaydedin.
df.to_csv(r'C:\Users\beyza\Desktop\AIN3007_project\test_annotations_and_slides.csv', index=False)
df.shape

100


(100, 3)

In [9]:
df.head()

,Slides,image_shape,annotations
0,D:/AIN3007_project/external_test_data/breast__...,"(65536, 46080)",D:/AIN3007_project/external_test_data/breast__...
1,D:/AIN3007_project/external_test_data/breast__...,"(37405, 140264)",D:/AIN3007_project/external_test_data/breast__...
2,D:/AIN3007_project/external_test_data/breast__...,"(57271, 129159)",D:/AIN3007_project/external_test_data/breast__...
3,D:/AIN3007_project/external_test_data/breast__...,"(63944, 107024)",D:/AIN3007_project/external_test_data/breast__...
4,D:/AIN3007_project/external_test_data/breast__...,"(45056, 44544)",D:/AIN3007_project/external_test_data/breast__...


In [10]:
for index, row in df.iterrows():
    image_shape = row['image_shape']
    if image_shape.startswith("(") and image_shape.endswith(")"):
        image_shape = image_shape[1:-1]  # Parantezleri kaldır
        dimensions = image_shape.split(',')
        if len(dimensions) == 2:
            dimensions[0], dimensions[1] = dimensions[1], dimensions[0]  # Boyutları değiştir
            new_image_shape = f"({dimensions[0]}, {dimensions[1]})"
            df.at[index, 'image_shape'] = new_image_shape

def process_image_shape(x):
    try:
        # Parantezleri ve boşlukları kaldırın, sonra virgüllerle ayırın
        parts = x.strip('()').split(',')
        # Her iki parçayı da tamsayıya dönüştürün
        return tuple(map(int, parts))
    except ValueError:
        # Hata durumunda, None döndürün veya başka bir işlem yapın
        return None

# image_shape sütununu işleyin ve hatalı girdileri filtreleyin
df['image_shape'] = df['image_shape'].apply(process_image_shape)

df.to_csv(r'C:\Users\beyza\Desktop\AIN3007_project\test_annotations_and_slides.csv', index=False)
df.head()

,Slides,image_shape,annotations
0,D:/AIN3007_project/external_test_data/breast__...,"(46080, 65536)",D:/AIN3007_project/external_test_data/breast__...
1,D:/AIN3007_project/external_test_data/breast__...,"(140264, 37405)",D:/AIN3007_project/external_test_data/breast__...
2,D:/AIN3007_project/external_test_data/breast__...,"(129159, 57271)",D:/AIN3007_project/external_test_data/breast__...
3,D:/AIN3007_project/external_test_data/breast__...,"(107024, 63944)",D:/AIN3007_project/external_test_data/breast__...
4,D:/AIN3007_project/external_test_data/breast__...,"(44544, 45056)",D:/AIN3007_project/external_test_data/breast__...


In [11]:
import xml.etree.ElementTree as ET

def read_xml(xml_file_path):
    # Parse the XML file
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    return root

def extract_coordinates(xml_root):
    # Initialize dictionaries to hold coordinates for each group
    groups = {
        "roi": [],
        "tissue": [],
        "bg": []
    }
    
    # Iterate through all the 'Annotation' elements
    for annotation in xml_root.findall('.//Annotation'):
        part_of_group = annotation.get('PartOfGroup')  # Get the group name
        coords = []
        
        # For each 'Coordinate' in 'Coordinates', extract 'X' and 'Y'
        for coordinate in annotation.find('Coordinates'):
            x = float(coordinate.get('X').replace(',', '.'))  # Ensure number format
            y = float(coordinate.get('Y').replace(',', '.'))
            coords.append((x, y))
        
        # Add the coordinate list to the appropriate group based on 'PartOfGroup'
        if part_of_group in groups:
            groups[part_of_group].append(coords)
    
    return groups

In [12]:
df['coords_list'] = None 
df['bg_coords'] = None
df['roi_coords'] = None
# Loop through the specified rows of the DataFrame
for index, row in df.iterrows():
    xml_file_path = row["annotations"]  # Get the path to the XML file
    xml_root = read_xml(xml_file_path)  # Read the XML file
    grouped_coordinates = extract_coordinates(xml_root)  # Extract and group coordinates

    # Assign the extracted coordinates to the DataFrame
    df.at[index, 'roi_coords'] = grouped_coordinates["roi"]
    df.at[index, 'coords_list'] = grouped_coordinates["tissue"]
    df.at[index, 'bg_coords'] = grouped_coordinates["bg"]    
df.to_csv(r'C:\Users\beyza\Desktop\AIN3007_project\test_annotations_and_slides.csv', index=False)


In [13]:
df.head()

,Slides,image_shape,annotations,coords_list,bg_coords,roi_coords
0,D:/AIN3007_project/external_test_data/breast__...,"(46080, 65536)",D:/AIN3007_project/external_test_data/breast__...,"[[(53404.7969, 14011.96), (53554.1875, 14046.4...",[],[]
1,D:/AIN3007_project/external_test_data/breast__...,"(140264, 37405)",D:/AIN3007_project/external_test_data/breast__...,"[[(26862.4258, 15002.2354), (26945.1113, 15072...","[[(18883.4824, 43299.625), (18890.3398, 43306....",[]
2,D:/AIN3007_project/external_test_data/breast__...,"(129159, 57271)",D:/AIN3007_project/external_test_data/breast__...,"[[(7929.19287, 2330.91797), (8071.69434, 2717....",[],[]
3,D:/AIN3007_project/external_test_data/breast__...,"(107024, 63944)",D:/AIN3007_project/external_test_data/breast__...,"[[(42510.0469, 19094.4121), (42834.1406, 19094...","[[(60135.6406, 7363.04883), (60194.6133, 7368....",[]
4,D:/AIN3007_project/external_test_data/breast__...,"(44544, 45056)",D:/AIN3007_project/external_test_data/breast__...,"[[(40301.3008, 7525.21094), (40353.0781, 7755....","[[(13763.8037, 25946.3516), (13809.6377, 26125...",[]


In [14]:
def create_binary_mask(roi_coords, tissue_coords, bg_coords, image_shape):
    # Ensure image_shape is a tuple of integers
    image_shape = tuple(map(int, image_shape))

    # Define the image shape based on roi_coords
    if roi_coords:
        top_left = roi_coords[0][0]  # (x_min, y_min)
        bottom_right = roi_coords[0][2]  # (x_max, y_max)
        image_shape = (int(bottom_right[1] - top_left[1]), int(bottom_right[0] - top_left[0]))  # (height, width)
    else:
        top_left = (0, 0)  # (x_min, y_min)
        bottom_right = (image_shape[1], image_shape[0])  # (x_max, y_max)
        image_shape = (int(bottom_right[1] - top_left[1]), int(bottom_right[0] - top_left[0]))  # (height, width)

    # Rest of your code remains the same

    # Initialize an all-white image (background)
    binary_mask = np.full(image_shape, 255, dtype=np.uint8)

    # Draw the 'tissue' areas in black
    for coords in tissue_coords:
        # Convert coordinates to relative positions within the ROI
        relative_coords = [(int(x - top_left[0]), int(y - top_left[1])) for x, y in coords]
        pts = np.array(relative_coords, np.int32).reshape((-1, 1, 2))
        cv2.fillPoly(binary_mask, [pts], 0)  # 0 for black

    # Draw the 'bg' areas in white
    for coords in bg_coords:
        # Convert coordinates to relative positions within the ROI
        relative_coords = [(int(x - top_left[0]), int(y - top_left[1])) for x, y in coords]
        pts = np.array(relative_coords, np.int32).reshape((-1, 1, 2))
        cv2.fillPoly(binary_mask, [pts], 255)  # 255 for white

    return binary_mask

In [15]:
output_dir = 'C:\\Users\\beyza\\Desktop\\AIN3007_project\\test_mask'
os.makedirs(output_dir, exist_ok=True)

# Iterate through the specified range of the DataFrame
for index, row in df.iloc[0:100].iterrows():
    # Extract coordinates from the DataFrame row
    roi_coords = row['roi_coords']
    tissue_coords = row['coords_list']
    bg_coords = row['bg_coords']
    image_shape = row['image_shape']
   
   # Create the binary mask
    binary_mask = create_binary_mask(roi_coords, tissue_coords, bg_coords, image_shape)
    
    # Define the path to save the binary mask
    mask_filename = os.path.join(output_dir, f'mask_{index}.png')
    cv2.imwrite(mask_filename, binary_mask)
    print(f"Mask for row {index} saved.")

Mask for row 0 saved.
Mask for row 1 saved.
Mask for row 2 saved.
Mask for row 3 saved.
Mask for row 4 saved.
Mask for row 5 saved.
Mask for row 6 saved.
Mask for row 7 saved.
Mask for row 8 saved.
Mask for row 9 saved.
Mask for row 10 saved.
Mask for row 11 saved.
Mask for row 12 saved.
Mask for row 13 saved.
Mask for row 14 saved.
Mask for row 15 saved.
Mask for row 16 saved.
Mask for row 17 saved.
Mask for row 18 saved.
Mask for row 19 saved.
Mask for row 20 saved.
Mask for row 21 saved.
Mask for row 22 saved.
Mask for row 23 saved.
Mask for row 24 saved.
Mask for row 25 saved.
Mask for row 26 saved.
Mask for row 27 saved.
Mask for row 28 saved.
Mask for row 29 saved.
Mask for row 30 saved.
Mask for row 31 saved.
Mask for row 32 saved.
Mask for row 33 saved.
Mask for row 34 saved.
Mask for row 35 saved.
Mask for row 36 saved.
Mask for row 37 saved.
Mask for row 38 saved.
Mask for row 39 saved.
Mask for row 40 saved.
Mask for row 41 saved.
Mask for row 42 saved.
Mask for row 43 saved

In [20]:
mask_folder_path = "C:/Users/beyza/Desktop/AIN3007_project/test_mask/"

# Mask dosyalarının sayısı
num_masks = len(df)

# Mask dosyalarının yollarını eklemek için bir liste oluşturma
mask_paths = [os.path.join(mask_folder_path, f"mask_{i}.png") for i in range(num_masks)]

# Yeni "mask_path" sütununu veri çerçevesine ekleme
df["mask_path"] = mask_paths
df.to_csv(r'C:\Users\beyza\Desktop\AIN3007_project\test_annotations_and_slides.csv', index=False)
df.head()

,Slides,image_shape,annotations,coords_list,bg_coords,roi_coords,mask_path
0,D:/AIN3007_project/external_test_data/breast__...,"(46080, 65536)",D:/AIN3007_project/external_test_data/breast__...,"[[(53404.7969, 14011.96), (53554.1875, 14046.4...",[],[],C:/Users/beyza/Desktop/AIN3007_project/test_ma...
1,D:/AIN3007_project/external_test_data/breast__...,"(140264, 37405)",D:/AIN3007_project/external_test_data/breast__...,"[[(26862.4258, 15002.2354), (26945.1113, 15072...","[[(18883.4824, 43299.625), (18890.3398, 43306....",[],C:/Users/beyza/Desktop/AIN3007_project/test_ma...
2,D:/AIN3007_project/external_test_data/breast__...,"(129159, 57271)",D:/AIN3007_project/external_test_data/breast__...,"[[(7929.19287, 2330.91797), (8071.69434, 2717....",[],[],C:/Users/beyza/Desktop/AIN3007_project/test_ma...
3,D:/AIN3007_project/external_test_data/breast__...,"(107024, 63944)",D:/AIN3007_project/external_test_data/breast__...,"[[(42510.0469, 19094.4121), (42834.1406, 19094...","[[(60135.6406, 7363.04883), (60194.6133, 7368....",[],C:/Users/beyza/Desktop/AIN3007_project/test_ma...
4,D:/AIN3007_project/external_test_data/breast__...,"(44544, 45056)",D:/AIN3007_project/external_test_data/breast__...,"[[(40301.3008, 7525.21094), (40353.0781, 7755....","[[(13763.8037, 25946.3516), (13809.6377, 26125...",[],C:/Users/beyza/Desktop/AIN3007_project/test_ma...


In [16]:
df.iloc[0:50]

,Slides,image_shape,annotations,coords_list,bg_coords,roi_coords
0,D:/AIN3007_project/external_test_data/breast__...,"(46080, 65536)",D:/AIN3007_project/external_test_data/breast__...,"[[(53404.7969, 14011.96), (53554.1875, 14046.4...",[],[]
1,D:/AIN3007_project/external_test_data/breast__...,"(140264, 37405)",D:/AIN3007_project/external_test_data/breast__...,"[[(26862.4258, 15002.2354), (26945.1113, 15072...","[[(18883.4824, 43299.625), (18890.3398, 43306....",[]
2,D:/AIN3007_project/external_test_data/breast__...,"(129159, 57271)",D:/AIN3007_project/external_test_data/breast__...,"[[(7929.19287, 2330.91797), (8071.69434, 2717....",[],[]
3,D:/AIN3007_project/external_test_data/breast__...,"(107024, 63944)",D:/AIN3007_project/external_test_data/breast__...,"[[(42510.0469, 19094.4121), (42834.1406, 19094...","[[(60135.6406, 7363.04883), (60194.6133, 7368....",[]
4,D:/AIN3007_project/external_test_data/breast__...,"(44544, 45056)",D:/AIN3007_project/external_test_data/breast__...,"[[(40301.3008, 7525.21094), (40353.0781, 7755....","[[(13763.8037, 25946.3516), (13809.6377, 26125...",[]
5,D:/AIN3007_project/external_test_data/breast__...,"(55808, 49152)",D:/AIN3007_project/external_test_data/breast__...,"[[(28587.582, 12472.1475), (28522.0352, 12664....","[[(23717.7441, 42536.2617), (23714.7324, 42596...",[]
6,D:/AIN3007_project/external_test_data/breast__...,"(45568, 53248)",D:/AIN3007_project/external_test_data/breast__...,"[[(18935.1406, 4957.13281), (19022.1094, 5023....","[[(30037.0801, 28890.4355), (30062.0059, 29001...",[]
7,D:/AIN3007_project/external_test_data/breast__...,"(171563, 63809)",D:/AIN3007_project/external_test_data/breast__...,"[[(58944.2695, 20104.5781), (58979.0938, 20127...","[[(18284.0449, 89097.4375), (18316.6992, 89113...",[]
8,D:/AIN3007_project/external_test_data/breast__...,"(80128, 40960)",D:/AIN3007_project/external_test_data/breast__...,"[[(14763.8213, 4268.11523), (14977.4873, 4236....","[[(10670.2002, 4098.66846), (10746.3184, 4118....",[]
9,D:/AIN3007_project/external_test_data/breast__...,"(56064, 57344)",D:/AIN3007_project/external_test_data/breast__...,"[[(41433.0195, 31782.1309), (41498.8203, 31321...","[[(42003.8984, 40233.0078), (42282.1836, 40443...",[]


In [17]:
df.iloc[50:100]

,Slides,image_shape,annotations,coords_list,bg_coords,roi_coords
50,D:/AIN3007_project/external_test_data/prostate...,"(30720, 7680)",D:/AIN3007_project/external_test_data/prostate...,"[[(6384.0, 22512.0), (6368.0, 22528.0), (6192....",[],[]
51,D:/AIN3007_project/external_test_data/prostate...,"(28672, 9984)",D:/AIN3007_project/external_test_data/prostate...,"[[(9216.0, 27792.0), (9200.0, 27808.0), (9168....",[],[]
52,D:/AIN3007_project/external_test_data/prostate...,"(26536, 46017)",D:/AIN3007_project/external_test_data/prostate...,"[[(14272.0, 19456.0), (14272.0, 19504.0), (142...",[],[]
53,D:/AIN3007_project/external_test_data/prostate...,"(48074, 73818)",D:/AIN3007_project/external_test_data/prostate...,"[[(45824.0, 28160.0), (45824.0, 28192.0), (457...",[],[]
54,D:/AIN3007_project/external_test_data/prostate...,"(35012, 51769)",D:/AIN3007_project/external_test_data/prostate...,"[[(16832.0, 24320.0), (16832.0, 24368.0), (168...",[],[]
55,D:/AIN3007_project/external_test_data/prostate...,"(4608, 13568)",D:/AIN3007_project/external_test_data/prostate...,"[[(3248.0, 512.0), (3232.0, 528.0), (3184.0, 5...","[[(1608.75195, 1783.73901), (1615.02393, 1808....",[]
56,D:/AIN3007_project/external_test_data/prostate...,"(35244, 60397)",D:/AIN3007_project/external_test_data/prostate...,"[[(40192.0, 27392.0), (40192.0, 27440.0), (401...",[],[]
57,D:/AIN3007_project/external_test_data/prostate...,"(5120, 4352)",D:/AIN3007_project/external_test_data/prostate...,"[[(769.610291, 4093.81812), (709.195801, 4120....","[[(3229.42212, 2245.16284), (3241.58936, 2230....",[]
58,D:/AIN3007_project/external_test_data/prostate...,"(29578, 62314)",D:/AIN3007_project/external_test_data/prostate...,"[[(14976.0, 15680.0), (14976.0, 15728.0), (149...",[],[]
59,D:/AIN3007_project/external_test_data/prostate...,"(32166, 56562)",D:/AIN3007_project/external_test_data/prostate...,"[[(7168.0, 24192.0), (7168.0, 24240.0), (7152....",[],[]


In [25]:
from PIL import Image
import numpy as np
from openslide import OpenSlide
import os
from patchify import patchify

Image.MAX_IMAGE_PIXELS = None  

def find_appropriate_level(wsi, desired_patch_size, desired_patch_number):
    """
    WSI'daki en uygun seviyeyi bulur.
    desired_patch_size: Oluşturulacak patch'in piksel cinsinden boyutu
    desired_patch_number: Oluşturulmak istenen minimum patch sayısı
    """
    for level in range(wsi.level_count - 1, -1, -1):
        width, height = wsi.level_dimensions[level]
        if (width // desired_patch_size) * (height // desired_patch_size) >= desired_patch_number:
            return level
    return 0  # Eğer uygun seviye bulunamazsa en düşük seviyeyi kullan

def crop_patches(wsi_path, mask_path, desired_patch_size=256, desired_patch_number=100):
    wsi = OpenSlide(wsi_path)
    mask = Image.open(mask_path)
    
    # En uygun seviyeyi bul
    level = find_appropriate_level(wsi, desired_patch_size, desired_patch_number)

    downsampled_image = wsi.read_region((0, 0), level, wsi.level_dimensions[level])
    image_array = np.array(downsampled_image.convert("RGB"))
    mask_array = np.array(mask)

    # Resize the mask to match the WSI's downsampled dimensions
    mask_array_resized = np.array(mask.resize(wsi.level_dimensions[level]))

    # Define patch size
    patch_size = 256

    # Calculate padding needed for both dimensions
    def calculate_padding(array, patch_size):
        pad_height = patch_size - (array.shape[0] % patch_size)
        pad_width = patch_size - (array.shape[1] % patch_size)
        return pad_height, pad_width

    image_pad_height, image_pad_width = calculate_padding(image_array, patch_size)
    mask_pad_height, mask_pad_width = calculate_padding(mask_array_resized, patch_size)

    # Pad the arrays
    image_array_padded = np.pad(image_array, ((0, image_pad_height), (0, image_pad_width), (0, 0)), mode='constant', constant_values=255)
    mask_array_padded = np.pad(mask_array_resized, ((0, mask_pad_height), (0, mask_pad_width)), mode='constant', constant_values=255)

    # Create patches for both image and mask
    image_patches = patchify(image_array_padded, (patch_size, patch_size, 3), step=patch_size)
    mask_patches = patchify(mask_array_padded, (patch_size, patch_size), step=patch_size)

    # Extract the filename part from the WSI path
    wsi_filename_part = os.path.basename(wsi_path).split('.')[0]

    # Specify the base directory for saving patches
    base_save_dir = "C:/Users/beyza/Desktop/AIN3007_project/test_patch_crop/"
    image_dir = os.path.join(base_save_dir, f'image_{wsi_filename_part}')
    mask_dir = os.path.join(base_save_dir, f'mask_{wsi_filename_part}')

    # Ensure the directories for image and mask patches are created
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)

    # Save the patches
    for i in range(image_patches.shape[0]):
        for j in range(image_patches.shape[1]):
            # Save image patch
            image_patch = image_patches[i, j, 0, :, :, :]
            image_patch_img = Image.fromarray(image_patch)
            image_patch_path = os.path.join(image_dir, f'patch_{i}_{j}.png')
            image_patch_img.save(image_patch_path)

            # Save mask patch
            mask_patch = mask_patches[i, j, :, :]
            mask_patch_img = Image.fromarray(mask_patch)
            mask_patch_path = os.path.join(mask_dir, f'patch_{i}_{j}.png')
            mask_patch_img.save(mask_patch_path)


In [ ]:
for index, row in df.iloc[90:100].iterrows():
    wsi_path = row["Slides"]
    mask_path = row["mask_path"]
    crop_patches(wsi_path, mask_path) 

In [23]:
def find_appropriate_level(wsi, desired_patch_size, desired_patch_number):
    """
    WSI'daki en uygun seviyeyi bulur.
    desired_patch_size: Oluşturulacak patch'in piksel cinsinden boyutu
    desired_patch_number: Oluşturulmak istenen minimum patch sayısı
    """
    for level in range(wsi.level_count - 1, -1, -1):
        width, height = wsi.level_dimensions[level]
        if (width // desired_patch_size) * (height // desired_patch_size) >= desired_patch_number:
            return level
    return 0  


def crop_patches(wsi_path, mask_path, roi_coords, desired_patch_size=256, desired_patch_number=100):
    wsi = OpenSlide(wsi_path)
    mask = Image.open(mask_path)

    level = find_appropriate_level(wsi, desired_patch_size, desired_patch_number)

    if roi_coords:
        # Extracting top-left and bottom-right coordinates
        top_left = roi_coords[0][0]  
        top_left_int = (int(top_left[0]), int(top_left[1]))
        bottom_right = roi_coords[0][2]  

        # Calculating width and height of the ROI
        roi_width = bottom_right[0] - top_left[0]
        roi_height = bottom_right[1] - top_left[1]

        # Adjust ROI dimensions according to the level downsampling
        level_downsample = wsi.level_downsamples[level]
        x, y = int(top_left[0] / level_downsample), int(top_left[1] / level_downsample)
        width, height = int(roi_width / level_downsample), int(roi_height / level_downsample)

        # Extend the ROI if necessary to make it divisible by the patch size
        patch_size = 256
        width_extended = (width // patch_size + (width % patch_size > 0)) * patch_size
        height_extended = (height // patch_size + (height % patch_size > 0)) * patch_size

        downsampled_image = wsi.read_region(top_left_int, level, (width_extended, height_extended))
    else:
        downsampled_image = wsi.read_region((0, 0), level, wsi.level_dimensions[level])

    # Convert the PIL images to numpy arrays
    image_array = np.array(downsampled_image)
    mask_array = np.array(mask)

    # Ensure the mask is resized to match the WSI's downsampled dimensions
    mask_array_resized = np.array(mask.resize(downsampled_image.size))

    # Create patches for both image and mask
    image_patches = patchify(image_array, (patch_size, patch_size, 3), step=patch_size)
    mask_patches = patchify(mask_array_resized, (patch_size, patch_size), step=patch_size)
    
    wsi_filename_part = os.path.basename(wsi_path).split('.')[0]
    
    base_save_dir = "C:/Users/beyza/Desktop/AIN3007_project/test_patch_crop/"
    image_dir = os.path.join(base_save_dir, f'image_{wsi_filename_part}')
    mask_dir = os.path.join(base_save_dir, f'mask_{wsi_filename_part}')
    
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)
    
    for i in range(image_patches.shape[0]):
        for j in range(image_patches.shape[1]):
            # Save image patch
            image_patch = image_patches[i, j, 0, :, :, :]
            image_patch_img = Image.fromarray(image_patch)
            image_patch_path = os.path.join(image_dir, f'patch_{i}_{j}.png')
            image_patch_img.save(image_patch_path)

            # Save mask patch
            mask_patch = mask_patches[i, j, :, :]
            mask_patch_img = Image.fromarray(mask_patch)
            mask_patch_path = os.path.join(mask_dir, f'patch_{i}_{j}.png')
            mask_patch_img.save(mask_patch_path)


In [24]:
for index, row in df.iloc[80:90].iterrows():
    wsi_path = row["Slides"]
    mask_path = row["mask_path"]
    roi_coords = row["roi_coords"]
    crop_patches(wsi_path, mask_path, roi_coords) 